## Install Lib

In [1]:
import os

!git clone https://github.com/openai/shap-e.git  > /dev/null 2>&1
%cd shap-e/
!pip install -e . > /dev/null 2>&1
!pip install numpy-stl > /dev/null 2>&1
!apt-get install blender > /dev/null 2>&1

!sudo apt-get update > /dev/null 2>&1
!sudo apt-get install xvfb  > /dev/null 2>&1
os.environ["PATH"] = "/usr/bin/xvfb:" + os.environ["PATH"]

!wget https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.3/blender-3.3.12-linux-x64.tar.xz > /dev/null 2>&1
!tar -xf blender-3.3.12-linux-x64.tar.xz

%cd /content/shap-e/blender-3.3.12-linux-x64

!./blender --version

/content/shap-e
/content/shap-e/blender-3.3.12-linux-x64
Blender 3.3.12
	build date: 2023-10-19
	build time: 08:43:00
	build commit date: 2023-10-17
	build commit time: 09:05
	build hash: 10ade9ceee90
	build platform: Linux
	build type: release
	build c flags:  -Wall -Werror=implicit-function-declaration -Werror=return-type -Werror=vla -Wstrict-prototypes -Wmissing-prototypes -Wno-char-subscripts -Wno-unknown-pragmas -Wpointer-arith -Wunused-parameter -Wwrite-strings -Wlogical-op -Wundef -Winit-self -Wmissing-include-dirs -Wno-div-by-zero -Wtype-limits -Wformat-signedness -Wrestrict -Wnonnull -Wabsolute-value -Wuninitialized -Wredundant-decls -Wshadow -Wno-error=unused-but-set-variable -Wimplicit-fallthrough=5  -std=gnu11 -msse -pipe -fPIC -funsigned-char -fno-strict-aliasing -ffp-contract=off -msse2 -D_GLIBCXX_USE_CXX11_ABI=0  
	build c++ flags:  -Wuninitialized -Wredundant-decls -Wall -Wno-invalid-offsetof -Wno-sign-compare -Wlogical-op -Winit-self -Wmissing-include-dirs -Wno-div-by-

In [2]:
blender_executable_path = "/content/shap-e/blender-3.3.12-linux-x64/blender"

!rm -f /usr/local/bin/blender

!ln -s {blender_executable_path} /usr/local/bin/blender

import os
os.environ['BLENDER_PATH'] = '/usr/local/bin/blender'

!echo $BLENDER_PATH

!blender --version

!{blender_executable_path} --version

/usr/local/bin/blender
Blender 3.3.12
	build date: 2023-10-19
	build time: 08:43:00
	build commit date: 2023-10-17
	build commit time: 09:05
	build hash: 10ade9ceee90
	build platform: Linux
	build type: release
	build c flags:  -Wall -Werror=implicit-function-declaration -Werror=return-type -Werror=vla -Wstrict-prototypes -Wmissing-prototypes -Wno-char-subscripts -Wno-unknown-pragmas -Wpointer-arith -Wunused-parameter -Wwrite-strings -Wlogical-op -Wundef -Winit-self -Wmissing-include-dirs -Wno-div-by-zero -Wtype-limits -Wformat-signedness -Wrestrict -Wnonnull -Wabsolute-value -Wuninitialized -Wredundant-decls -Wshadow -Wno-error=unused-but-set-variable -Wimplicit-fallthrough=5  -std=gnu11 -msse -pipe -fPIC -funsigned-char -fno-strict-aliasing -ffp-contract=off -msse2 -D_GLIBCXX_USE_CXX11_ABI=0  
	build c++ flags:  -Wuninitialized -Wredundant-decls -Wall -Wno-invalid-offsetof -Wno-sign-compare -Wlogical-op -Winit-self -Wmissing-include-dirs -Wno-div-by-zero -Wtype-limits -Werror=return-

## Code

In [3]:
!blender --version

Blender 3.3.12
	build date: 2023-10-19
	build time: 08:43:00
	build commit date: 2023-10-17
	build commit time: 09:05
	build hash: 10ade9ceee90
	build platform: Linux
	build type: release
	build c flags:  -Wall -Werror=implicit-function-declaration -Werror=return-type -Werror=vla -Wstrict-prototypes -Wmissing-prototypes -Wno-char-subscripts -Wno-unknown-pragmas -Wpointer-arith -Wunused-parameter -Wwrite-strings -Wlogical-op -Wundef -Winit-self -Wmissing-include-dirs -Wno-div-by-zero -Wtype-limits -Wformat-signedness -Wrestrict -Wnonnull -Wabsolute-value -Wuninitialized -Wredundant-decls -Wshadow -Wno-error=unused-but-set-variable -Wimplicit-fallthrough=5  -std=gnu11 -msse -pipe -fPIC -funsigned-char -fno-strict-aliasing -ffp-contract=off -msse2 -D_GLIBCXX_USE_CXX11_ABI=0  
	build c++ flags:  -Wuninitialized -Wredundant-decls -Wall -Wno-invalid-offsetof -Wno-sign-compare -Wlogical-op -Winit-self -Wmissing-include-dirs -Wno-div-by-zero -Wtype-limits -Werror=return-type -Wno-char-subscrip

In [4]:
%cd /content/shap-e

/content/shap-e


In [8]:
!pip install torch torchvision  > /dev/null 2>&1

!pip install fvcore  > /dev/null 2>&1
#!pip install "git+https://github.com/facebookresearch/pytorch3d.git@stable"  > /dev/null 2>&1

!pip install trimesh  > /dev/null 2>&1
!pip install open3d  > /dev/null 2>&1

In [6]:
ls

blender-3.3.12-linux-x64/        LICENSE        README.md  samples.md  shap_e/
blender-3.3.12-linux-x64.tar.xz  model-card.md  samples/   setup.py    shap_e.egg-info/


In [9]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))
batch_size = 4
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("snake.jpeg")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:11<00:00, 83.4MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPb29fT09PTz8vPx7+/u7O7t6u7q5fLl2O3l2ezr5+vp4uzm2+…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAO7Z2/TQyfrLxfTLwvnIwvXIwvDHv+fGvvXEuu7CvPK+tvK6re…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPf29vb19fXz8vbw7fTw7fPv7fXt5/Lu6/Tq5O/q6PPm3+7m4P…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPv8+/r6+vn6+fn5+ff4+Pb39/X29fX19fTz8vLz8/Ly8vHw8P…

In [ ]:
xm = load_model('transmitter', device=device)
model_text = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

batch_size = 4
guidance_scale = 15.0
prompt = "a blue bird"

latents = sample_latents(
    batch_size=batch_size,
    model=model_text,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))


  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAFrB4Vm/4Fe931e83le73lS53Ve351S33le05Va05FS13FS121…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAGnA5We/5Wm+5Ge94mW942W74GS63mW34mS24WS14GG13mGz3V…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAKHQ4om70oTH5XW3znauyW+y0Gyty2Sj0lahwUeexGGQtFGUw0…

In [18]:
!pip install torch torchvision torchaudio > /dev/null 2>&1
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'  > /dev/null 2>&1

from shap_e.util.notebooks import (
    create_pan_cameras,
    decode_latent_images,
    decode_latent_mesh,
)

for i, latent in enumerate(latents):
    filename = f"/tmp/mesh_{i}.obj"
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(filename, "w") as f:
        t.write_obj(f)


/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


KeyboardInterrupt: ignored